# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import io
import requests
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model


from azureml.widgets import RunDetails
from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

experiment_name = 'fraud_detection'

exp=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print(exp)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AFLD56HN9 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-143035
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-143035
Experiment(Name: fraud_detection,
Workspace: quick-starts-ws-143035)


In [3]:
dataset = Dataset.get_by_name(workspace=ws, name='Banking-Transactions')
transactions_df = dataset.to_pandas_dataframe()

In [4]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    # counts for all classes
    counts = transactions_df['Class'].value_counts()
    
    # get fraudulent and valid cnts
    fraud_cnts = counts[1]
    valid_cnts = counts[0]
    
    # calculate percentage of fraudulent data
    fraud_percentage = fraud_cnts/(fraud_cnts+valid_cnts)
    
    return fraud_percentage

In [5]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(dataset)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transactions_df.shape[0])
print('Out of (total) pts: ', transactions_df.shape[0])

Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Create compute cluster
# Choose a name for the computer cluster:
amlcompute_cluster_name = "compute-cluster"

# Check if the cluster already exists:
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [7]:
# Set the automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Class",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [8]:
# Submit the experiment
remote_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
fraud_detection,AutoML_ca8f21f0-f9ec-4565-b576-7a7bca6608f7,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
# Retrieve and save your best automl model.
best_automl_run, best_model = remote_run.get_output()
best_automl_model_name = best_automl_run.properties['model_name']
print(('Best AutoML model name:{}\n\n' \
       'Best AutoML model run:{}\n\n' \
       'Best AutoML model specification:{}').format(best_automl_model_name, str(best_automl_run), str(best_model)))

Best AutoML model name:AutoMLca8f21f0f38

Best AutoML model run:Run(Experiment: fraud_detection,
Id: AutoML_ca8f21f0-f9ec-4565-b576-7a7bca6608f7_38,
Type: azureml.scriptrun,
Status: Completed)

Best AutoML model specification:Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                  min_samples_split=0.10368421052631578,
                             

In [22]:
registered_model =  remote_run.register_model(model_name = 'best_automl_model')

registered_model.download(target_dir='outputs', exist_ok=True)

'outputs/model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

Creating a pipeline

In [25]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                memory_gb=4,
                                                enable_app_insights=True,
                                                description='predict a fraud')
model = Model(ws, 'best_automl_model')

env_deploy = Environment.get(workspace=ws, name='AzureML-AutoML')

inference_config = InferenceConfig(entry_script='score.py', environment=env_deploy)

service = Model.deploy(workspace=ws,
                       name='fraud-detection3',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)

In [27]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
SucceededACI service creation operation finished, operation "Succeeded"


In [29]:
print(service.scoring_uri)
print(service.swagger_uri)

http://bb1b70be-3179-4c95-840d-22a8035a7223.southcentralus.azurecontainer.io/score
http://bb1b70be-3179-4c95-840d-22a8035a7223.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [31]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
# key = ''



# One set of data to score, so we get two results back
data = {
  "data": [
    {
      "Time": 472,
      "V1": -3.0435406239976,
      "V2": -3.15730712090228,
      "V3": 1.08846277997285,
      "V4": 2.2886436183814,
      "V5": 1.35980512966107,
      "V6": -1.06482252298131,
      "V7": 0.325574266158614,
      "V8": -0.0677936531906277,
      "V9": -0.270952836226548,
      "V10": -0.838586564582682,
      "V11": -0.414575448285725,
      "V12": -0.503140859566824,
      "V13": 0.676501544635863,
      "V14": -1.69202893305906,
      "V15": 2.00063483909015,
      "V16": 0.666779695901966,
      "V17": 0.599717413841732,
      "V18": 1.72532100745514,
      "V19": 0.283344830149495,
      "V20": 2.10233879259444,
      "V21": 0.661695924845707,
      "V22": 0.435477208966341,
      "V23": 1.37596574254306,
      "V24": -0.293803152734021,
      "V25": 0.279798031841214,
      "V26": -0.145361714815161,
      "V27": -0.252773122530705,
      "V28": 0.0357642251788156,
      "Amount": 529
    }
  ]
}
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

DataErrorException:
	Message: df should be a pandas dataframe or numpy array
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "SystemError",
        "message": "Encountered an internal AutoML error. Error Message/Code: df should be a pandas dataframe or numpy array. Additional Info: DataErrorException:\n\tMessage: df should be a pandas dataframe or numpy array\n\tInnerException: None\n\tErrorResponse \n{\n    \"error\": {\n        \"message\": \"df should be a pandas dataframe or numpy array\",\n        \"target\": \"utilities.check_input\",\n        \"reference_code\": \"098d83d9-f525-4844-9666-ea28b1a41f08\"\n    }\n}",
        "details_uri": "https://docs.microsoft.com/azure/machine-learning/resource-known-issues#automated-machine-learning",
        "target": "utilities.check_input",
        "inner_error": {
            "code": "ClientError",
            "inner_error": {
                "code": "AutoMLInternal"
            }
        },
        "reference_code":

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
print(service.get_logs())

In [ ]:
# delete service
service.delete()

In [ ]:
# delete compute cluster
cpu_cluster.delete()